In [167]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import dmanip, vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [168]:
# get data
data = pd.read_csv('Level_1_b_NonPath_Path_GN_2286.csv') 

# split data
X=data.iloc[:,1:151]
y=data.iloc[:,151]

#y=data.iloc[:,151].astype('category')

print(X)
print(y)

          Xc1.A      Xc1.R  ...  Grantham.Xd.29  Grantham.Xd.30
0      2.565095   2.331904  ...        0.032327        0.030319
1     15.716045  12.278160  ...        0.033492        0.033055
2     13.299348   5.269553  ...        0.031429        0.031817
3      7.756695   9.637106  ...        0.031756        0.032999
4      5.367732   2.044850  ...        0.031482        0.031596
...         ...        ...  ...             ...             ...
2281   3.434023   2.113245  ...        0.029650        0.032397
2282   8.597566   6.513308  ...        0.032343        0.028946
2283   8.597410   1.719482  ...        0.033358        0.028789
2284   5.471856   0.521129  ...        0.032696        0.027279
2285   5.471856   0.521129  ...        0.032696        0.027279

[2286 rows x 150 columns]
0       2
1       2
2       2
3       2
4       2
       ..
2281    1
2282    1
2283    1
2284    1
2285    1
Name: Class, Length: 2286, dtype: int64


In [169]:
# init
n_epochs = 10
n_comps = X.shape[1]

In [170]:
# encode the classification labels

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

#from collections import Counter
#from imblearn.over_sampling import SMOTE 


#sm = SMOTE(random_state=45)
#x_b, y_b = sm.fit_resample(X, yy)
#print('Resampled dataset shape %s' % Counter(Y_res))
# split the dataset
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.25,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

In [185]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128,activation='sigmoid', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64,activation='sigmoid', dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(2 ,activation='softmax'))




# print model architecture summary
print(model.summary())

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_78 (LSTM)               (None, 150, 150)          91200     
_________________________________________________________________
lstm_79 (LSTM)               (None, 150, 128)          142848    
_________________________________________________________________
lstm_80 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_53 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_54 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_55 (Dense)             (None, 2)                 130       
Total params: 300,162
Trainable params: 300,162
Non-trainable params: 0
_______________________________________________

In [186]:
# compile model
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [187]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=64,
                    validation_data=(x_test, y_test))

Epoch 1/10
27/27 [==============================] - 55s 2s/step - loss: 0.7055 - accuracy: 0.4963 - val_loss: 0.6922 - val_accuracy: 0.5227
Epoch 2/10
27/27 [==============================] - 39s 1s/step - loss: 0.6951 - accuracy: 0.4832 - val_loss: 0.6932 - val_accuracy: 0.5227
Epoch 3/10
27/27 [==============================] - 39s 1s/step - loss: 0.6904 - accuracy: 0.5329 - val_loss: 0.6927 - val_accuracy: 0.5227
Epoch 4/10
27/27 [==============================] - 39s 1s/step - loss: 0.6936 - accuracy: 0.5009 - val_loss: 0.6936 - val_accuracy: 0.5227
Epoch 5/10
27/27 [==============================] - 39s 1s/step - loss: 0.6941 - accuracy: 0.5107 - val_loss: 0.6921 - val_accuracy: 0.5227
Epoch 6/10
27/27 [==============================] - 39s 1s/step - loss: 0.6927 - accuracy: 0.5279 - val_loss: 0.6956 - val_accuracy: 0.4773
Epoch 7/10
27/27 [==============================] - 39s 1s/step - loss: 0.6933 - accuracy: 0.5098 - val_loss: 0.6928 - val_accuracy: 0.5227
Epoch 8/10
27/27 [==

In [178]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.5186697782963827
test-acc = 0.5227272727272727
train-CFR =               precision    recall  f1-score   support

           0       0.52      1.00      0.68       889
           1       0.00      0.00      0.00       825

    accuracy                           0.52      1714
   macro avg       0.26      0.50      0.34      1714
weighted avg       0.27      0.52      0.35      1714

test-CFR =               precision    recall  f1-score   support

           0       0.52      1.00      0.69       299
           1       0.00      0.00      0.00       273

    accuracy                           0.52       572
   macro avg       0.26      0.50      0.34       572
weighted avg       0.27      0.52      0.36       572



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
